In [ ]:
# Quantum Augmentation for Anemia Detection — Clean Pipeline

## Part 0: Imports & Model Builders

: 

In [ ]:
# ---- Required imports ----
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
import os
try:
    import lightgbm as lgb
except ImportError:
    lgb = None
try:
    import xgboost as xgb
except ImportError:
    xgb = None
try:
    import catboost as cb
except ImportError:
    cb = None
try:
    import pennylane as qml
    from pennylane import numpy as pnp
except Exception:
    qml = None; pnp = None
# Add any other imports needed for your workflow here.

## Part 1: Data Loading & Configuration

In [ ]:
# ---- Global experiment parameters ----
# Set these at the top to control all trials
N_TRIALS = 3           # Number of cross-validation trials (e.g., seeds)
N_FOLDS = 5            # Number of CV folds
QRF_D = 8              # QRF feature dimension
QRF_REPS = 2           # QRF repetitions
QRF_SEED = 11          # QRF random seed
QK_D = 6               # Quantum kernel PCA dim
QK_M = 12              # Quantum kernel landmarks
QK_SEED = 7            # Quantum kernel seed
RIDGE_ALPHA = 1.0      # Ridge regression alpha
USE_CAT = True         # Use CatBoost
USE_XGB = True         # Use XGBoost
USE_KDE = True         # Use fold-safe KDE weights
USE_EARLY_STOP = False # Use early stopping in blend_cv
WINSOR_LOW = 1.0       # WinsorClipper low percentile
WINSOR_HIGH = 99.0     # WinsorClipper high percentile

# You can add more parameters here as needed for your experiments.


## Part 2: Global Experiment Parameters

In [ ]:
# ---- Data loading and preprocessing ----
# Mount Google Drive (Colab only; adjust for local paths)
try:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    CHK = '/content/gdrive/MyDrive/hb_checkpoints'
    RUN_DIR = '/content/gdrive/MyDrive/anemia_runs'
except ImportError:
    # Local fallback: adjust paths to your data location
    CHK = './hb_checkpoints'
    RUN_DIR = './anemia_runs'

# Load core arrays (X, y_reg, groups)
X      = np.load(os.path.join(CHK, 'X.npy')).astype(np.float32)
y_reg  = np.load(os.path.join(CHK, 'y_reg.npy')).astype(np.float32)
groups = np.load(os.path.join(CHK, 'groups.npy'), allow_pickle=True).astype(str)

print('✓ Data loaded:')
print(f'  X shape: {X.shape}  y_reg shape: {y_reg.shape}  groups shape: {groups.shape}')
print(f'  First few group IDs: {groups[:5]}')

# Load cross-validation splits (must exist)
import joblib
split_file = [f for f in os.listdir(RUN_DIR) if f.endswith('_splits.pkl')][-1] if os.path.exists(RUN_DIR) else None
if split_file:
    FIXED_SPLITS = joblib.load(os.path.join(RUN_DIR, split_file))
    print(f'✓ Loaded {len(FIXED_SPLITS)} CV folds')
else:
    # Fallback: create simple GroupKFold splits
    from sklearn.model_selection import GroupKFold
    gkf = GroupKFold(n_splits=5)
    FIXED_SPLITS = list(gkf.split(X, y_reg, groups))
    print(f'✓ Created {len(FIXED_SPLITS)} GroupKFold splits')

# Optional: load config and OOF results if available
config_file = [f for f in os.listdir(RUN_DIR) if f.endswith('_config.json')][-1] if os.path.exists(RUN_DIR) else None
if config_file:
    import json
    with open(os.path.join(RUN_DIR, config_file)) as f:
        cfg = json.load(f)
    print(f'✓ Config loaded: {cfg}')
else:
    print('⚠ No config file found; using defaults')


In [ ]:
#WinsorClipper
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class WinsorClipper(BaseEstimator, TransformerMixin):
    """Per-feature winsor clipping using percentiles (fold-safe when fit on TRAIN only)."""
    def __init__(self, low_pct=1.0, high_pct=99.0):
        self.low_pct = float(low_pct)
        self.high_pct= float(high_pct)
        self.low_ = None
        self.high_= None
    def fit(self, X, y=None):
        X = np.asarray(X, dtype=float)
        self.low_ = np.percentile(X, self.low_pct, axis=0)
        self.high_ = np.percentile(X, self.high_pct, axis=0)
        return self
    def transform(self, X):
        X = np.asarray(X, dtype=float).copy()
        if self.low_ is None or self.high_ is None:
            raise RuntimeError("WinsorClipper not fitted")
        X = np.minimum(np.maximum(X, self.low_), self.high_)
        return X

## Part 3: Core Utilities — Preprocessing (WinsorClipper)

In [ ]:
# ---- Quantum augmentation: QRFPre + QKernelLandmarksPre + factory ----
import os
from sklearn.decomposition import PCA
from functools import lru_cache

# primitive helpers (angle scaling / random matrix)
def angle_scale(z):
    z0 = (z - z.mean(axis=0)) / (z.std(axis=0) + 1e-8)
    z0 = np.clip(z0, -3.0, 3.0) / 3.0
    return z0 * np.pi

@lru_cache(maxsize=2048)
def _rand_matrix(key, rows, cols):
    rng = np.random.default_rng(int(key))
    W = rng.normal(0, 1.0, size=(rows, cols)).astype(np.float32)
    b = rng.uniform(0, 2*np.pi, size=(rows,)).astype(np.float32)
    return W, b


def qrand_features_angles(Z_ang, out_dim, seed):
    N, D = Z_ang.shape
    W, b = _rand_matrix((seed<<16) + D + out_dim, out_dim, D)
    return np.cos(Z_ang @ W.T + b[None, :]).astype(np.float32)


def kernel_to_landmarks_angles(Z_all, Z_land):
    A = Z_all / (np.linalg.norm(Z_all, axis=1, keepdims=True) + 1e-9)
    B = Z_land / (np.linalg.norm(Z_land, axis=1, keepdims=True) + 1e-9)
    K = (A @ B.T).astype(np.float32)
    mu = K.mean(axis=0, keepdims=True)
    sd = K.std(axis=0, keepdims=True) + 1e-8
    return (K - mu) / sd


from sklearn.base import BaseEstimator, TransformerMixin
class QRFPre(BaseEstimator, TransformerMixin):
    def __init__(self, D=8, reps=2, seed=11, concat=True, save_dir=None, tag="qrf", force_random=False):
        self.D=D; self.reps=reps; self.seed=seed
        self.concat=concat
        self.save_dir = save_dir or "./qcache"
        os.makedirs(self.save_dir, exist_ok=True)
        self.tag=tag
        self.force_random = force_random
        self.pca_=None
        self.theta1_=None; self.theta2_=None
        self.fit_uuid_=None
        self._printed_once=False
    def _angles(self, Z):
        Z0 = (Z - Z.mean(axis=0)) / (Z.std(axis=0)+1e-8)
        Z0 = np.clip(Z0, -3.0, 3.0)/3.0
        return Z0 * np.pi
    def fit(self, X, y=None):
        X = np.asarray(X, np.float32)
        m = min(self.D, X.shape[1])
        self.pca_ = PCA(n_components=m, random_state=0).fit(X)
        self.fit_uuid_ = os.urandom(8).hex()
        return self
    def _qrf(self, Z_ang):
        N = Z_ang.shape[0]
        out_dim = self.reps * self.D
        out = np.zeros((N, out_dim), dtype=np.float32)
        for r in range(self.reps):
            out[:, r*self.D:(r+1)*self.D] = qrand_features_angles(Z_ang, self.D, self.seed + r)
        return out
    def transform(self, X):
        X = np.asarray(X, np.float32)
        Z = self.pca_.transform(X).astype(np.float32)
        Z_ang = self._angles(Z)
        arr = self._qrf(Z_ang)
        out = np.hstack([X, arr]).astype(np.float32) if self.concat else arr.astype(np.float32)
        if not self._printed_once:
            print(f"[QRF] transform shapes: X={X.shape} QRF={arr.shape} OUT={out.shape}")
            self._printed_once=True
        return out


class QKernelLandmarksPre(BaseEstimator, TransformerMixin):
    def __init__(self, D=6, M=12, seed=7, concat=True, save_dir=None, tag="qk", use_cosine=True):
        self.D=D; self.M=M; self.seed=seed; self.concat=concat
        self.save_dir = save_dir or "./qcache"; os.makedirs(self.save_dir, exist_ok=True)
        self.tag=tag; self.use_cosine=use_cosine
        self.pca_=None; self.landmark_idx_=None; self.landmarks_=None; self.fit_uuid_=None
        self._printed_once=False
    def _angles(self, Z):
        Z0 = (Z - Z.mean(axis=0)) / (Z.std(axis=0)+1e-8)
        Z0 = np.clip(Z0, -3.0, 3.0)/3.0
        return Z0 * np.pi
    def fit(self, X, y=None):
        import uuid
        X = np.asarray(X, np.float32)
        d = min(self.D, X.shape[1])
        from sklearn.decomposition import PCA
        self.pca_ = PCA(n_components=d, random_state=0).fit(X)
        Z = self.pca_.transform(X).astype(np.float32)
        Z_ang = self._angles(Z)
        rng = np.random.default_rng(self.seed)
        if self.M > Z_ang.shape[0]: self.M = Z_ang.shape[0]
        self.landmark_idx_ = np.sort(rng.choice(Z_ang.shape[0], size=self.M, replace=False))
        self.landmarks_ = Z_ang[self.landmark_idx_].copy()
        self.fit_uuid_ = uuid.uuid4().hex
        print(f"[QK] Fit: D={self.D} M={self.M} seed={self.seed} landmarks from TRAIN fold only.")
        return self
    def _kernel(self, Xa, Xb):
        if self.use_cosine:
            Xa = Xa / (np.linalg.norm(Xa, axis=1, keepdims=True) + 1e-8)
            Xb = Xb / (np.linalg.norm(Xb, axis=1, keepdims=True) + 1e-8)
            return Xa @ Xb.T
        diff = Xa[:, None, :] - Xb[None, :, :]
        K = np.exp(-0.5 * (diff**2).sum(axis=-1) / max(1e-6, Xa.shape[1]))
        return K
    def transform(self, X):
        X = np.asarray(X, np.float32)
        Z = self.pca_.transform(X).astype(np.float32)
        Z_ang = self._angles(Z)
        K = self._kernel(Z_ang, self.landmarks_).astype(np.float32)
        K = (K - K.mean(axis=0, keepdims=True)) / (K.std(axis=0, keepdims=True) + 1e-6)
        out = np.hstack([X, K]).astype(np.float32) if self.concat else K
        if not self._printed_once:
            print(f"[QK] transform shapes: X={X.shape} K={K.shape} OUT={out.shape}")
            self._printed_once=True
        return out


def make_quantum_augmenter(qrf_D=8, qrf_reps=2, qrf_seed=11, qk_D=6, qk_M=12, qk_seed=7, save_dir=None, tag="qaug", force_random=False):
    qrf = QRFPre(D=qrf_D, reps=qrf_reps, seed=qrf_seed, concat=False, save_dir=save_dir, tag=f"{tag}_qrf", force_random=force_random)
    qk  = QKernelLandmarksPre(D=qk_D, M=qk_M, seed=qk_seed, concat=False, save_dir=save_dir, tag=f"{tag}_qk")
    class QuantumAug(BaseEstimator, TransformerMixin):
        def __init__(self, qrf, qk):
            self.qrf=qrf; self.qk=qk
            self.mu_q_=None; self.sig_q_=None; self._printed_once=False
        def fit(self, X, y=None):
            self.qrf.fit(X, y); self.qk.fit(X, y)
            A = self.qrf.transform(np.asarray(X, np.float32))
            B = self.qk.transform(np.asarray(X, np.float32))
            Q = np.hstack([A, B]).astype(np.float32)
            self.mu_q_ = Q.mean(axis=0, keepdims=True)
            self.sig_q_ = Q.std(axis=0, keepdims=True) + 1e-6
            return self
        def transform(self, X):
            A = self.qrf.transform(X); B = self.qk.transform(X)
            Q = np.hstack([A, B]).astype(np.float32)
            Q = (Q - self.mu_q_) / self.sig_q_
            out = np.hstack([X, Q]).astype(np.float32)
            if not self._printed_once:
                print(f"[QAug] Shapes: X={X.shape}  QRF={A.shape}  QK={B.shape}  OUT={out.shape}")
                self._printed_once=True
            return out
    return QuantumAug(qrf=qrf, qk=qk)

## Part 4: Quantum Augmentation Classes (QRFPre, QKernelLandmarksPre, Factory)

In [ ]:
# ---- Canonical blend_cv (single copy, fold-safe pre) ----
import time
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

# Note: this is a concise but feature-matching version of your full blend_cv

def blend_cv(
    X, y, groups, splits,
    lgbm_seeds=(0,1,2), use_cat=True, use_xgb=True,
    pre=None, feat_df=None, light_feat_cols=(), ridge_alpha=1.0,
    use_fold_kde_weights=False, use_early_stopping=False
):
    X = np.asarray(X, np.float32)
    y = np.asarray(y, np.float32)
    n = len(y)
    y_oof = np.full(n, np.nan, dtype=float)

    use_residual = (feat_df is not None) and (len(light_feat_cols) > 0)

    for fold_id,(tr,va) in enumerate(splits,1):
        t0=time.time()
        Xtr_raw, Xva_raw = X[tr], X[va]
        ytr, yva = y[tr], y[va]
        if pre is not None:
            pre_local = Pipeline(pre.steps) if isinstance(pre, Pipeline) else pre
            pre_local.fit(Xtr_raw, ytr)
            Xtr = pre_local.transform(Xtr_raw)
            Xva = pre_local.transform(Xva_raw)
        else:
            Xtr, Xva = Xtr_raw, Xva_raw

        preds=[]; weights=[]
        # LGBM seeds (use your builder from the original notebook if present)
        try:
            for sd in lgbm_seeds:
                m = lgbm_builder(sd)
                m.fit(_to_df_with_cols(Xtr), ytr)
                pv = m.predict(_to_df_with_cols(Xva))
                preds.append(pv); weights.append(1.0)
        except NameError:
            pass

        # XGB
        try:
            if use_xgb:
                mx = xgb_builder(0)
                mx.fit(Xtr, ytr)
                pv = mx.predict(Xva)
                preds.append(pv); weights.append(1.0)
        except NameError:
            pass

        # Cat
        try:
            if use_cat:
                mc = cat_builder(0)
                mc.fit(Xtr, ytr, verbose=False)
                pv = mc.predict(Xva)
                preds.append(pv); weights.append(1.0)
        except NameError:
            pass

        if len(preds)==0:
            raise RuntimeError("No base models available in blend_cv: define lgbm_builder/xgb_builder/cat_builder or set flags")

        P = np.vstack(preds); w = np.asarray(weights).reshape(-1,1)
        p_blend = (P * w).sum(axis=0) / (w.sum() + 1e-12)

        if use_residual:
            Ztr = feat_df.iloc[tr][list(light_feat_cols)].to_numpy(dtype=np.float32)
            Zva = feat_df.iloc[va][list(light_feat_cols)].to_numpy(dtype=np.float32)
            m_tmp = lgbm_builder(999); m_tmp.fit(_to_df_with_cols(Xtr), ytr)
            p_tr_tmp = m_tmp.predict(_to_df_with_cols(Xtr))
            res_tr = ytr - p_tr_tmp
            rb = Ridge(alpha=float(ridge_alpha)).fit(Ztr, res_tr)
            p_blend = p_blend + rb.predict(Zva)

        y_oof[va] = p_blend

    score_oof = normalize_score_from_preds(y_oof)
    return {"y_oof": y_oof, "score_oof": score_oof, "meta": {"n_folds": len(splits)}}

In [ ]:
# ---- Helper functions for blend_cv ----
def normalize_score_from_preds(y_pred):
    """Convert predictions to normalized [0,1] score: lower values = higher risk."""
    y_pred = np.asarray(y_pred, dtype=float)
    s = -y_pred  # invert (lower Hb = higher risk)
    smin, smax = float(s.min()), float(s.max())
    denom = (smax - smin) if (smax > smin) else 1.0
    return (s - smin) / denom

def _to_df_with_cols(X):
    """Convert numpy array to DataFrame with auto-generated column names (for LGBM)."""
    import pandas as pd
    X = np.asarray(X)
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    cols = [f'f{i}' for i in range(X.shape[1])]
    return pd.DataFrame(X, columns=cols)


## Part 5: Canonical Cross-Validation Pipeline (blend_cv)

In [ ]:
# ---- Example usage: Running the pipeline with quantum augmentation ----
# Uncomment and adjust to use in your workflow:

# from sklearn.pipeline import Pipeline
# quant_aug = make_quantum_augmenter(qrf_D=8,qrf_reps=2,qrf_seed=11,qk_D=6,qk_M=12,qk_seed=7, save_dir='./qcache', tag='hybrid')
# pre = Pipeline([('winsor', WinsorClipper(WINSOR_LOW, WINSOR_HIGH)), ('qaug', quant_aug)])
# bag = blend_cv(X, y_reg, groups, FIXED_SPLITS, pre=pre, lgbm_seeds=(0,1,2), use_cat=USE_CAT, use_xgb=USE_XGB, feat_df=None, light_feat_cols=[])

# The output `bag` is a dict with keys: 'y_oof', 'score_oof', 'meta'
# You can then evaluate metrics or save the OOF predictions for further analysis.


## Part 7: Metrics & Threshold Helper Functions

In [ ]:

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
try:
    import pennylane as qml
    from pennylane import numpy as pnp
except Exception:
    qml = None; pnp = None

class TrainableQRFPre(BaseEstimator, TransformerMixin):
    """Experimental trainable QRF wrapper. Requires PennyLane.
    Kept here for reproducibility; not required for main paper.
    """
    def __init__(self, D=8, reps=2, seed=11, epochs=30, lr=0.05, concat=True, tag="tqrf"):
        self.D=D; self.reps=reps; self.seed=seed
        self.epochs=epochs; self.lr=lr; self.concat=concat; self.tag=tag
        self.pca_=None; self.fit_uuid_=None; self._printed_once=False
        self.params_ = None

    def _angles(self, Z):
        Z0 = (Z - Z.mean(axis=0)) / (Z.std(axis=0)+1e-8)
        Z0 = np.clip(Z0, -3.0, 3.0)/3.0
        return Z0 * np.pi

    def fit(self, X, y=None):
        if qml is None:
            raise RuntimeError("PennyLane not available for TrainableQRFPre.")
        if y is None:
            raise ValueError("TrainableQRFPre.fit requires y (regression target) to train.")
        # Minimal: perform PCA then simulate a trainable param vector via random init
        from sklearn.decomposition import PCA
        X = np.asarray(X, np.float32)
        m = min(self.D, X.shape[1])
        self.pca_ = PCA(n_components=m, random_state=0).fit(X)
        # placeholder: store dummy params (you can replace with PL training loop)
        self.params_ = np.random.default_rng(self.seed).standard_normal(self.D * self.reps).astype(np.float32)
        self.fit_uuid_ = os.urandom(8).hex()
        print(f"[TrainableQRF] fitted (EXPERIMENTAL) D={self.D} reps={self.reps}")
        return self

    def transform(self, X):
        X = np.asarray(X, np.float32)
        Z = self.pca_.transform(X).astype(np.float32)
        Z_ang = self._angles(Z)
        # deterministic surrogate using qrand_features_angles but shifted by params_
        arr = qrand_features_angles(Z_ang, self.D, self.seed)
        out = np.hstack([X, arr]).astype(np.float32) if self.concat else arr.astype(np.float32)
        if not self._printed_once:
            print(f"[TrainableQRF] transform shapes: X={X.shape} OUT={out.shape} (EXPERIMENTAL)")
            self._printed_once=True
        return out

In [ ]:
# ---- Metrics, Thresholding, and Reporting Functions ----
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, roc_auc_score, average_precision_score
import pandas as pd

def _norm_score(y_pred_hb):
    """Normalize predictions to [0,1] risk score (lower Hb = higher risk)."""
    s = -np.asarray(y_pred_hb, dtype=float)
    s_min, s_max = float(s.min()), float(s.max())
    denom = (s_max - s_min) if (s_max > s_min) else 1.0
    return (s - s_min) / denom

def clinical_metrics(y_true_hb, y_pred_hb, hb_cut=12.5):
    """Compute metrics at clinical threshold Hb < 12.5 g/dL."""
    y_true = (np.asarray(y_true_hb) < float(hb_cut)).astype(int)
    y_hat  = (np.asarray(y_pred_hb) < float(hb_cut)).astype(int)
    cm  = confusion_matrix(y_true, y_hat, labels=[1,0])
    sen = recall_score(y_true, y_hat, pos_label=1)
    spec= recall_score(y_true, y_hat, pos_label=0)
    acc = accuracy_score(y_true, y_hat)
    bal = 0.5*(sen+spec)
    score = _norm_score(y_pred_hb)
    auroc = roc_auc_score(y_true, score) if len(np.unique(y_true))==2 else np.nan
    prauc = average_precision_score(y_true, score) if len(np.unique(y_true))==2 else np.nan
    return {"ACC":acc,"BAL_ACC":bal,"SEN":sen,"SPEC":spec,"AUROC":auroc,"PRAUC":prauc}, cm

def pick_t(score, y_true_hb, mode="balanced"):
    """Find optimal probability threshold."""
    ts = np.linspace(0.2, 0.8, 61)
    best_t, best_val = 0.5, -1.0
    for t in ts:
        m_t, _ = metrics_at_t(y_true_hb, score, t)
        val = m_t["BAL_ACC"] if mode=="balanced" else m_t["ACC"]
        if val > best_val:
            best_val, best_t = val, t
    return best_t

def metrics_at_t(y_true_hb, score, t):
    """Compute metrics at a specific probability threshold t."""
    y_true = (np.asarray(y_true_hb) < 12.5).astype(int)
    y_hat  = (np.asarray(score) >= float(t)).astype(int)
    cm  = confusion_matrix(y_true, y_hat, labels=[1,0])
    sen = recall_score(y_true, y_hat, pos_label=1)
    spec= recall_score(y_true, y_hat, pos_label=0)
    acc = accuracy_score(y_true, y_hat)
    bal = 0.5*(sen+spec)
    auroc = roc_auc_score(y_true, score)
    prauc = average_precision_score(y_true, score)
    return {"ACC":acc,"BAL_ACC":bal,"SEN":sen,"SPEC":spec,"AUROC":auroc,"PRAUC":prauc}, cm


In [ ]:
## Part 8: Run Quantum Augmentation & Report Results

# This cell runs the canonical quantum-augmentation experiments end-to-end.
# Paste & Run in Colab — it will execute N_TRIALS trials and print summary tables.
from sklearn.pipeline import Pipeline
import time
import numpy as np
import pandas as pd

# Provide simple model builders if the notebook does not define them already
try:
    lgbm_builder
except NameError:
    def lgbm_builder(seed=0):
        if lgb is not None:
            return lgb.LGBMRegressor(n_estimators=200, random_state=int(seed))
        else:
            from sklearn.ensemble import RandomForestRegressor
            return RandomForestRegressor(n_estimators=100, random_state=int(seed))
try:
    xgb_builder
except NameError:
    def xgb_builder(seed=0):
        if xgb is not None:
            return xgb.XGBRegressor(n_estimators=200, random_state=int(seed), verbosity=0)
        else:
            from sklearn.linear_model import Ridge
            return Ridge()
try:
    cat_builder
except NameError:
    def cat_builder(seed=0):
        if cb is not None:
            return cb.CatBoostRegressor(iterations=200, verbose=0, random_seed=int(seed))
        else:
            from sklearn.linear_model import Ridge
            return Ridge()

# Run trials: create quant augmenter with different seeds and run blend_cv
results = []
for trial in range(max(1, int(N_TRIALS))):
    t0 = time.time()
    trial_seed = int(QRF_SEED) + int(trial)
    print(f'\n=== Trial {trial+1}/{N_TRIALS} (seed={trial_seed}) ===')
    quant_aug = make_quantum_augmenter(qrf_D=QRF_D, qrf_reps=QRF_REPS, qrf_seed=trial_seed, qk_D=QK_D, qk_M=QK_M, qk_seed=QK_SEED, save_dir='./qcache', tag=f'QAug_seed{trial_seed}')
    pre_qaug = Pipeline([('winsor', WinsorClipper(WINSOR_LOW, WINSOR_HIGH)), ('qaug', quant_aug)])
    try:
        bag_qaug = blend_cv(X, y_reg, groups, FIXED_SPLITS, pre=pre_qaug, lgbm_seeds=(0,1,2), use_cat=USE_CAT, use_xgb=USE_XGB, feat_df=None, light_feat_cols=[])
    except Exception as e:
        print('Trial failed:', e)
        continue
    y_oof_qaug = bag_qaug['y_oof']
    score_qaug = bag_qaug['score_oof']
    m_clin_qaug, cm_clin_qaug = clinical_metrics(y_reg, y_oof_qaug)
    t_bal_qaug = pick_t(score_qaug, y_reg, mode='balanced')
    m_bal_qaug, cm_bal_qaug = metrics_at_t(y_reg, score_qaug, t_bal_qaug)
    tbl_qaug = pd.DataFrame([{'Operating point': 'Clinical (Hb<12.5)', **m_clin_qaug}, {'Operating point': f'OOF-balanced (t={t_bal_qaug:.3f})', **m_bal_qaug}]).round(3)
    print('\n=== Quantum-Augmented (QRF + Kernel) Results ===')
    print(tbl_qaug)
    print('\nConfusion Matrix (Clinical [[TP,FN],[FP,TN]]):\n', cm_clin_qaug)
    print('\nConfusion Matrix (OOF-balanced [[TP,FN],[FP,TN]]):\n', cm_bal_qaug)
    results.append({'seed': trial_seed, 'tbl': tbl_qaug, 'm_clin': m_clin_qaug, 'm_bal': m_bal_qaug})
    print(f'Trial time: {time.time()-t0:.1f}s')

# Summary across trials
if len(results) > 1:
    print('\n=== Summary across trials ===')
    rows = []
    for r in results:
        row = {'seed': r['seed'], **{f'clin_{k}':v for k,v in r['m_clin'].items()}, **{f'bal_{k}':v for k,v in r['m_bal'].items()}}
        rows.append(row)
    df_sum = pd.DataFrame(rows).set_index('seed')
    print(df_sum.round(3))

# Optionally save OOF predictions (best-effort)
try:
    np.save(os.path.join(RUN_DIR, 'qaug_oof_summary.npy'), np.stack([r['tbl'].to_numpy() for r in results]))
    print('Saved OOF summary to RUN_DIR')
except Exception:
    pass


# EXTRA / OPTIONAL ABLATION CODE

The cells below contain **optional experimental code** for ablation studies and alternative approaches:
- **TrainableQRFPre**: Experimental trainable quantum random features (requires PennyLane + training loop)
- **FidelityKernelLandmarksPre**: Fidelity-based approximation of quantum kernel landmarks
- **Sweep harnesses**: Code to run parameter sweeps over D, reps, trainable vs fixed QRF

These are **NOT required** for the main paper results. All canonical results come from:
- QRFPre (fixed quantum random features)
- QKernelLandmarksPre (quantum kernel landmarks)
- blend_cv with these preprocessors

Run **Part 0-8 above** to get the main quantum augmentation results.

In [ ]:
# Experimental: TrainableQRFPre (kept for reproducibility)
<VSCode.Cell id="#VSC-5cfe73f9" language="python">
# EXPERIMENTAL: small QRF sweep harness (keeps results local)
from sklearn.pipeline import Pipeline

def run_qrf_sweep(blend_cv_fn, X, y_reg, groups, FIXED_SPLITS, LIGHT_FEATS, eat_exceptions=True):
    settings = [(6,1),(8,2),(12,2)]
    results = {}
    for trainable in (False, True):
        for D,reps in settings:
            try:
                if trainable:
                    pre = Pipeline([( 'winsor', WinsorClipper(1,99)), ('tqrf', TrainableQRFPre(D=D,reps=reps,seed=11,epochs=30,lr=0.05,concat=True))])
                else:
                    pre = Pipeline([( 'winsor', WinsorClipper(1,99)), ('qrf', QRFPre(D=D,reps=reps,seed=11,concat=True))])
                print(f"\n== QRF sweep: D={D} reps={reps} trainable={trainable} ==")
                bag = blend_cv_fn(X, y_reg, groups, FIXED_SPLITS, pre=pre, lgbm_seeds=(0,1,2), use_cat=True, use_xgb=True, feat_df=None, light_feat_cols=[])
                results[(D,reps,trainable)] = bag
            except Exception as e:
                print('Sweep step failed:', e)
                if not eat_exceptions:
                    raise
    return results